In [14]:
import json

MODEL_TYPE = "c"
LAG = 4
PRED_NEXT = 2
THRESHOLD = 0.5
LOWER_BOUND = 0
UPPER_BOUND = 1
MODEL_NAME = f"{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}"

In [15]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import SSMContainer
from jesse.helpers import date_to_timestamp

from pathlib import Path
import lightgbm as lgb
import pandas as pd
import numpy as np

T_START = "2025-01-01"
T_END = "2025-10-20"

path_features = Path(".").parent / "strategies" / "BinanceBtcDeapV1Voting" / "models" / "feature_info.json"
with open(path_features) as f:
    feature_info: dict[str, list[str]] = json.load(f)
FEAT_FRACDIFF: list[str] = feature_info["fracdiff"]
ALL_RAW_FEAT = []
ALL_RAW_FEAT.extend(FEAT_FRACDIFF)
ALL_RAW_FEAT.extend(feature_info[MODEL_NAME])
ALL_RAW_FEAT = set(ALL_RAW_FEAT)
ALL_RAW_FEAT = sorted(
    [
        i
        for i in ALL_RAW_FEAT
        if not i.startswith("deep_ssm") and not i.startswith("lg_ssm")
    ]
)

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles(T_START, T_END)

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp(T_START)) &
    (df_feat_full.index.to_numpy() < date_to_timestamp(T_END))
]

getting raw features...
df_raw_feats.shape = (3103, 653)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json


In [18]:
with open("strategies/BinanceBtcDeapV1Voting/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

close_price = candles[:, 2][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")
assert len(datetimes) == len(close_price)

model = lgb.Booster(model_file=f"strategies/BinanceBtcDeapV1Voting/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [19]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    close_price,
    PRED_NEXT,
    output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
    granularity=0.02,
    lower_bound=LOWER_BOUND,
    upper_bound=UPPER_BOUND,
    threshold=THRESHOLD,
)


📊 时序对齐信息:
  - pred_next: 2 (预测未来第2根K线)
  - 原始数据长度: 3103
  - 对齐后数据长度: 3101
  - 丢弃的尾部数据: 2 条

已生成: slice_0.9800_1.0000_ratio_0.0103.jpg (切片区间: [0.9800, 1.0000] (做多), 样本占比: 1.03%)
已生成: slice_0.9600_0.9800_ratio_0.0281.jpg (切片区间: [0.9600, 0.9800] (做多), 样本占比: 2.81%)
已生成: slice_0.9400_0.9600_ratio_0.0303.jpg (切片区间: [0.9400, 0.9600] (做多), 样本占比: 3.03%)
已生成: slice_0.9200_0.9400_ratio_0.0187.jpg (切片区间: [0.9200, 0.9400] (做多), 样本占比: 1.87%)
已生成: slice_0.9000_0.9200_ratio_0.0226.jpg (切片区间: [0.9000, 0.9200] (做多), 样本占比: 2.26%)
已生成: slice_0.8800_0.9000_ratio_0.0245.jpg (切片区间: [0.8800, 0.9000] (做多), 样本占比: 2.45%)
已生成: slice_0.8600_0.8800_ratio_0.0181.jpg (切片区间: [0.8600, 0.8800] (做多), 样本占比: 1.81%)
已生成: slice_0.8400_0.8600_ratio_0.0168.jpg (切片区间: [0.8400, 0.8600] (做多), 样本占比: 1.68%)
已生成: slice_0.8200_0.8400_ratio_0.0184.jpg (切片区间: [0.8200, 0.8400] (做多), 样本占比: 1.84%)
已生成: slice_0.8000_0.8200_ratio_0.0203.jpg (切片区间: [0.8000, 0.8200] (做多), 样本占比: 2.03%)
已生成: slice_0.7800_0.8000_ratio_0.0177.jpg (切片区间: [0.7800,

In [36]:
from strategies.BinanceBtcDeapV1Voting.models.config import LGBMContainer

mc = LGBMContainer(
    model_type=MODEL_TYPE,
    lag=LAG,
    pred_next=PRED_NEXT,
    threshold=THRESHOLD,
)
print(mc.MODEL_NAME)

c_L3_N2


In [32]:
# 根据图片设定合适的filters

GIVEUP_FILTERS = [
    (0, 0.02),
    (0.12, 0.14),
    (0.24, 0.26),
    (0.32, 0.38),
    (0.4, 0.42),
    (0.46, 0.48),
    (0.5, 0.52),
]

REVERSE_FILTERS = [
    (0.04, 0.08),
    (0.16, 0.18),
    (0.2, 0.24),
    (0.28, 0.3),
    (0.48, 0.5),
    (0.52, 0.54),
]

for f in GIVEUP_FILTERS:
    mc.add_giveup_filter(f[0], f[1])

for f in REVERSE_FILTERS:
    mc.add_reverse_filter(f[0], f[1])

In [34]:
mc.save_filters()

Filters saved to /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/model_c_L3_N1_filters.json
